## 03. Computer Vision with PyTorch

What we're going to cover:
* Basic dataset
    * PyTorch FashionMNIST - https://pytorch.org/vision/stable/datasets.html
* Custom dataset 
    * (made from scratch)
* Visualizing models